From the 3 CSV files, we will begin cleaing and filtering the data

In [1]:
# import libraries
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
# import geonamescache
from collections import defaultdict

# function definitions
def calculate_age(born):
    born = datetime.strptime(born, '%Y-%m-%d').date()
    today = date.today()
    if (today.month, today.year) < (born.month, born.year):
        age = (today.year - born.year) - 1
    else:
        age = today.year - born.year
        
    return age

# assign csv data urls
# anime_data    = 'https://f000.backblazeb2.com/file/mal-db/AnimeList.csv'
# user_data     = 'https://f000.backblazeb2.com/file/mal-db/UserList.csv'
# user_mal_data = 'https://f000.backblazeb2.com/file/mal-db/UserAnimeList.csv'

# local directory
anime_data    = 'C:\\Users\\Uri\\Desktop\\data\\AnimeList.csv'
user_data     = 'C:\\Users\\Uri\\Desktop\\data\\UserList.csv'
user_mal_data = 'C:\\Users\\Uri\\Desktop\\data\\UserAnimeList.csv'

cities = 'worldcities.csv'

# set maximum number of rows to 20 & define NaN identifiers
pd.set_option('max_rows', 20)
idntfrs = ['na', '-', '--', '?', 'None', 'none', 'non', '', ' ', \
           'Not available', '0']

# read & import data into pandas data frames
anime_df    = pd.read_csv(anime_data, na_values=idntfrs)
user_df     = pd.read_csv(user_data, na_values=idntfrs)
cities_df   = pd.read_csv(cities)

# my_reader = pd.read_csv(user_mal_data, chunksize=my_chunk, iterator=True)
# user_mal_df = pd.concat(my_reader, ignore_index=True)

# since UserAnimeList.csv is too large, read it from a generator in chunks
# user_mal_gen = pd.read_csv(user_mal_data, na_values=idntfrs, iterator=True, \
#                            chunksize = my_chunk)
# user_mal_df  = next(user_mal_gen)

# display shape and rows of each data frame
# print('anime_df Shape:', anime_df.shape)
# anime_df.head()

# print('user_df Shape:', user_df.shape)
# user_df.head()

# print('user_mal_df Shape:', user_mal_df.shape)
# user_mal_df.head()

# drop unwanted features from the data frames
anime_df.drop(['title_english', 'title_japanese', 'title_synonyms', \
 			   'image_url', 'type', 'source', 'episodes', 'airing', 'aired', \
 			   'duration', 'rating', 'broadcast', 'related', \
 			   'producer', 'licensor', 'premiered', 'studio', 'opening_theme', \
 			   'ending_theme', 'background', 'favorites'],
               axis=1, inplace=True)

user_df.drop(['user_watching', 'user_completed', 'user_onhold', 'user_dropped', \
              'user_plantowatch', 'user_days_spent_watching', 'access_rank', \
              'join_date', 'last_online', 'stats_mean_score', 'stats_rewatched', \
              'stats_episodes'],
               axis=1, inplace=True)

# cities_df.drop(['\"city\"', '\"lat\"', '\"lng\"', '\"iso2\"', '\"iso3\"' \
#                 '\"admin_name\"', '\"capital\"', '\"population\"', '\"id\"'],
#                  axis=1, inplace=True)


cities_df.drop(['city', 'lat', 'lng', 'iso2', 'iso3', \
                'admin_name', 'capital', 'population', 'id'],
                 axis=1, inplace=True)
cities_df.rename(columns={'city_ascii': 'city'}, inplace=True)
cities_df['city'] = cities_df.city.str.lower()



for column in cities_df[['city']]:
    column_obj = cities_df[column]
    print(column_obj.values)













# drop all N/A values from the data frames
anime_df.dropna(inplace=True)
user_df.dropna(inplace=True)

# fix broken apostrophes across the entire dataframe
# anime_df.replace('&#039;', '\'', inplace=True)
anime_df['title'] = anime_df['title'].str.replace('&#039;', '\'')

# clean birth_date column so that it represents age as a number
user_df['age'] = user_df['birth_date'].apply(calculate_age)
user_df.drop('birth_date', axis=1, inplace=True)

# clean location column so that it only list country
# gc = geonamescache.GeonamesCache()
# countries = gc.get_countries()
# print(countries)





# remove animes that have not yet aired since they don't have scoring data
# anime_df = anime_df[~anime_df['status'].isin(['Not yet aired'])]

# remove NSFW content
# anime_df = anime_df[~anime_df['genre'].astype(str).str.contains('Hentai')]

# convert genres to a list
# anime_df['genre'] = anime_df.genre.str.split(',')

# write cleaned data frames to csv files
# anime_df.to_csv('anime.csv', index=False)
# user_df.to_csv('user.csv', index=False)

#print('anime_df Shape:', anime_df.shape)
#anime_df.head()
#anime_df.isna().sum()

#print('user_df Shape:', user_df.shape)
#user_df.head()
# user_df.isna().sum()

# print('user_mal_df Shape:', user_mal_df.shape)
# user_mal_df.head()
# user_mal_df.isna().sum()

# define chunk size for user_mal_data since the file is too large
# my_chunk = 10**5
# first_chunk = True
# for chunk in pd.read_csv(user_mal_data, na_values=idntfrs,
#                          chunksize=my_chunk, iterator=True):
#     user_mal_df = chunk
#     user_mal_df.drop(['my_watched_episodes', 'my_status', 'my_rewatching', \
#                       'my_rewatching_ep', 'my_last_updated', 'my_tags', \
#                       'my_start_date', 'my_finish_date'],
#                       axis=1, inplace=True)
#     user_mal_df.dropna(inplace=True)
#     if first_chunk:   
#         user_mal_df.to_csv('user_mal.csv', mode='w', header=True, index=False)
#         first_chunk = False
#     else:
#         user_mal_df.to_csv('user_mal.csv', mode='a', header=False, index=False)


print('cities_df Shape:', cities_df.shape)
cities_df.head()




['tokyo' 'jakarta' 'delhi' ... 'cheremoshna' 'ambarchik' 'nordvik']
cities_df Shape: (26569, 2)


,city,country
0,tokyo,Japan
1,jakarta,Indonesia
2,delhi,India
3,mumbai,India
4,manila,Philippines
